In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv020649/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
word = ("big","Data","Super","INTRESTING") 

In [4]:
wordsrdd = spark.sparkContext.parallelize(word)

In [5]:
# how many partition the rDD have, let 1 gb file ib HDFS, base RD loaded to RDD, then how many partition it will hwve
# 8 block 8 partition in memeory
wordsrdd.getNumPartitions()

2

In [6]:
spark.sparkContext.defaultParallelism
# set by admins

2

In [7]:
orderRdd = spark.sparkContext.textFile("/public/trendytech/orders/orders.csv")

In [8]:
orderRdd.getNumPartitions()

28

In [9]:
spark.sparkContext.defaultMinPartitions

2

# countByValue (o/p on driver)
below example will be for the understanding
its a action not a transformation = map+reduce
its only be used if its end, if more xfr is needed than map+reduce by key should be used, which is accross the cluster.

In [10]:
orderRdd = spark.sparkContext.textFile("/public/trendytech/orders/orders.csv")

In [11]:
orderRdd.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [12]:
mappedRdd = orderRdd.map(lambda x: (x.split(",")[3]))
mappedRdd.take(5)

['CLOSED', 'PENDING_PAYMENT', 'COMPLETE', 'CLOSED', 'COMPLETE']

In [13]:
# getting the same value 
# orderSplit.map(lambda x: (x[3],1)) + orderPerStatus1.reduceByKey(lambda x,y: x+y)
mappedRdd.countByValue()

defaultdict(int,
            {'CLOSED': 9445000,
             'PENDING_PAYMENT': 18787500,
             'COMPLETE': 28623750,
             'PROCESSING': 10343750,
             'PAYMENT_REVIEW': 911250,
             'PENDING': 9512500,
             'ON_HOLD': 4747500,
             'CANCELED': 1785000,
             'SUSPECTED_FRAUD': 1947500})